In [ ]:
import pandas as pd
import sqlite3

### Chinook Sample Database https://github.com/lerocha/chinook-database
### SQLite https://www.sqlite.org/
### SQLite Python https://docs.python.org/3/library/sqlite3.html

### Connect to the database

In [ ]:
con = sqlite3.connect("/content/Chinook_Sqlite.sqlite")

### List tables

In [ ]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

### Read the artist table into a dataframe

In [ ]:
df = pd.read_sql_query("SELECT * FROM artist", con)

### show last rows of artist

In [ ]:
df.tail()

### Add a new row to artist

In [ ]:
df.loc[275] = ({'ArtistId': 276, 'Name': 'GCC Orchestra'})
# df = df.drop(275)
df.tail()

### Save the artist dataframe back to the database table

In [ ]:
df.to_sql("Artist", con, if_exists="replace", index=False)

### Read the Playlist table into a dataframe

In [ ]:
playlist = pd.read_sql_query("SELECT * FROM Playlist", con)
playlist.head()

### Read the PlaylistTrack table into a dataframe

In [ ]:
track = pd.read_sql_query("SELECT * FROM PlaylistTrack", con)
track.head()

### Merge the track and playlist dataframes
https://www.w3schools.com/sql/sql_join_left.asp

https://pandas.pydata.org/docs/reference/api/pandas.merge.html

In [ ]:
pd.merge(track, playlist, on='PlaylistId', how='left')

### groupby the merged dataframes and sort

In [ ]:
pd.merge(track, playlist, on='PlaylistId', how='left').groupby('Name').count().sort_values(by='TrackId', ascending=False)


### Equivalent SQL join query

In [ ]:
sql = '''select p.Name , count(*) as Tracks
      from PlaylistTrack as pt
      left join playlist as p on p.PlaylistId = pt.PlaylistId
      group by p.Name
      order by Tracks desc'''
top_playlists = pd.read_sql_query(sql, con)
top_playlists

### Save the results of the query into a new table

In [ ]:
top_playlists.to_sql("top_playlists", con, if_exists="replace")
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

### Drop the newly created table

In [ ]:
sql = '''drop table top_playlists'''
con.execute(sql)

### List tables

In [ ]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

### Close the database connection

In [ ]:
con.close()

### Convert the notebook to html and submit the html file in canvas for grading

In [ ]:
# !jupyter nbconvert --to html